In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

from astropy import units as u
from astropy import coordinates as coords
from astroquery.simbad import Simbad
import astroquery
plt.style.use("fivethirtyeight")

In [2]:
import gwb

In [3]:
tgas = gwb.TGASData("/Users/semyeong/data/gaia/tgas_source/stacked_tgas.fits")

In [4]:
df = pd.read_csv("../data/targets.csv")
print(len(df))

45


In [218]:
c = coords.SkyCoord(df.tgas_ra, df.tgas_dec, unit='deg')

In [11]:
customSimbad = Simbad()
customSimbad.add_votable_fields('sptype')
customSimbad.add_votable_fields('orv')
customSimbad.add_votable_fields('bibcodelist(1850-2017)')

In [12]:
for field in ['rv_value', 'rvel', 'gcrv', 'orv']:
    customSimbad.get_field_description(field)

Radial velocity value. Eventually translated from a redshift
Radial velocities of galaxies
General Catalogue of Radial Velocities
oRV stellar radial velocities (will shortly replace 
              the GCRV measurements.


In [13]:
result = customSimbad.query_objects(df.hip_tyc_id)

In [ ]:
result['MAIN_ID', 'oRV_RVel',
 'oRV_Q',
 'oRV_Nmes',
 'oRV_Rem',
 'oRV_Orig',
 'oRV_Dis',
 'oRV_bibcode',]

In [16]:
result.sort('BIBLIST_1850_2017')
result['MAIN_ID', 'SP_TYPE', 'BIBLIST_1850_2017']

/Users/semyeong/anaconda2/envs/gwb/lib/python3.5/site-packages/astropy/table/column.py:1096: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
  ma.MaskedArray.__setitem__(self, index, value)


MAIN_ID,SP_TYPE,BIBLIST_1850_2017
object,object,int32
TYC 4164-274-1,,0
TYC 3851-600-1,,0
TYC 3867-1373-1,,0
TYC 3861-1374-1,,0
TYC 4173-609-1,,0
TYC 4174-1117-1,,0
HD 238351,G0,0
TYC 3877-725-1,,0
TYC 3860-1483-1,,0


In [6]:
import sys
sys.path.append('../../gaia-wide-binaries/notebooks/')
import tap

class SimbadTAP(tap.TAP_Service):
    def __init__(self, *args, **kwargs):
        host = "simbad.u-strasbg.fr"
        port = 80
        path = "/simbad/sim-tap"
        tap.TAP_Service.__init__(self, host, path, port, *args, **kwargs)

In [7]:
query_template = """
SELECT BIBCode,
       Journal,
       Title,
       Year,
       Volume,
       DOI
FROM ref JOIN has_ref ON oidbibref = oidbib JOIN ident ON has_ref.oidref = ident.oidref
WHERE id = '{}'
ORDER BY Year DESC;
"""

In [8]:
mytap = SimbadTAP()

In [20]:
mytap.get_table_info('ref')

oidbib,bibcode,year,journal,page,last_page,volume,title,doi,nbobject
int64,object,int16,object,int32,int32,int32,object,object,int32


In [9]:
import time

In [16]:
biblist = []
for iden in result['MAIN_ID']:
    print('searching for {}'.format(iden.decode()))
    biblist.append(mytap.query(query_template.format(iden.decode())))
    time.sleep(5)  ## seems that I'm violating some rate limit or something

searching for TYC 4164-274-1
searching for TYC 3471-333-1
searching for BD+51  1966
searching for HR  5345
searching for TYC 3489-1148-1
searching for TYC 3470-485-1
searching for HD 234061
searching for TYC 3868-177-1
searching for HD 238423
searching for BD+56  1762
searching for HD 234065
searching for TYC 3490-1083-1
searching for HD 133909
searching for TYC 3471-233-1
searching for TYC 3486-1405-1
searching for TYC 3860-1483-1
searching for TYC 3877-725-1
searching for HD 129865
searching for TYC 4174-1117-1
searching for TYC 4173-609-1
searching for HD 135501
searching for TYC 3867-1373-1
searching for BD+60  1587
searching for HR  5280
searching for TYC 3875-762-1
searching for NAME Stella Ludoviciana
searching for HD 113491
searching for TYC 4180-573-1
searching for *  81 UMa
searching for HD 124370
searching for HD 137589
searching for TYC 3851-600-1
searching for TYC 3861-1374-1
searching for HD 238351
searching for *  84 UMa
searching for HD 125557
searching for HD 145145
se

In [38]:
len(biblist), len(df), len(result)

(45, 45, 45)

In [39]:
# add which object's reference it is
for i, t in enumerate(biblist):
    t['hip_tyc_id'] = [df.hip_tyc_id[i] for j in range(len(t))]
    t['simbad_main_id'] = [result['MAIN_ID'][i] for j in range(len(t))]

In [40]:
from astropy import table

In [57]:
dfBib = pd.concat([t.to_pandas() for t in biblist])

In [43]:
# table.Table.from_pandas(dfBib).show_in_browser()

In [55]:
print("total number of refs = {:d}".format(len(dfBib)))
print("total number of unique refs = {:d}".format(dfBib.bibcode.unique().size))
print("number of stars with refs = {:d} / {:d}".format(
        df.hip_tyc_id.size - dfBib.hip_tyc_id.unique().size, df.hip_tyc_id.size))

total number of refs = 391
total number of unique refs = 232
number of stars with refs = 25 / 45


In [58]:
dfBib['bibcode'] = dfBib.bibcode.str.decode('utf-8')
dfBib['journal'] = dfBib.journal.str.decode('utf-8')
dfBib['title'] = dfBib.title.str.decode('utf-8')
dfBib['doi'] = dfBib.doi.str.decode('utf-8')
dfBib['simbad_main_id'] = dfBib.simbad_main_id.str.decode('utf-8')
dfBib['year'] = dfBib.year.apply(int)
dfBib['volume'] = dfBib.volume.apply(int)

In [59]:
dfBib.to_csv("../data/target_bib.csv", index=False)

In [79]:
irexcess = []
for name, g in dfBib.groupby('hip_tyc_id'):
    if '2012MNRAS.427..343M' in g.bibcode.values:
        irexcess.append(name)
    for title, bibcode in zip(g.title, g.bibcode):
        if 'x-ray' in title.lower():
            print(name, title, bibcode)
        

HIP 68637 Near infrared observations of soft X-ray selected AGN. 2002A&A...386..854G
HIP 69275 X-ray-emitting stars identified from the ROSAT All-Sky Survey and the  Sloan Digital Sky Survey. 2009ApJS..181..444A
HIP 69958 The X-ray emission of A-type stars. 1995PASP..107.1034S
TYC 3867-2-1 X-ray-emitting stars identified from the ROSAT All-Sky Survey and the  Sloan Digital Sky Survey. 2009ApJS..181..444A
TYC 3867-2-1 XID II: statistical cross-association of ROSAT bright source catalog  X-ray sources with 2MASS point source catalog near-infrared sources. 2009ApJS..184..138H
TYC 3867-2-1 Spectral classification of optical counterparts to ROSAT all-sky survey X-ray sources. 2007AJ....133.2495D
TYC 3867-2-1 The Hamburg/RASS Catalogue of optical identifications. Northern high-galactic latitude ROSAT Bright Source Catalogue X-ray  sources. 2003A&A...406..535Z
TYC 3867-281-1 X-ray-emitting stars identified from the ROSAT All-Sky Survey and the  Sloan Digital Sky Survey. 2009ApJS..181..444A


In [ ]:
xray = ['HIP 69275']

In [81]:
print('\n'.join(irexcess))

HIP 63702
HIP 66198
HIP 67005
HIP 67231
HIP 68637
HIP 69275
HIP 69650
HIP 69721
HIP 69917
HIP 69958
HIP 71911
HIP 72389
HIP 73730
HIP 74458
HIP 75449
HIP 77903
HIP 78958


In [ ]:
for name